# Modelo de Identificação de Fraudes

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config import DADOS_MASTERCARD_TRATADO

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_MASTERCARD_TRATADO)

In [4]:
# Verificar se todos os valores podem ser convertidos para inteiro (sem valores decimais)

if (transacoes['Time'] == transacoes['Time'].astype(int)).all():
    
    # Se todos os valores são inteiros, realizar a conversão
    
    transacoes['Time'] = transacoes['Time'].astype(int)
   
else:
    print("Nem todos os valores na coluna 'Time' podem ser convertidos para inteiro.")

### Vamos ajustar a escala de duas colunas para testar o resultado
- As colunas Time e Amount têm valores max muito alto. Destoa do restante do dataset.

In [6]:
# Primeiro para a coluna Time

transacoes.Time = transacoes.Time / transacoes.Time.max()

# E então para a coluna Amount

transacoes.Amount = transacoes.Amount / transacoes.Amount.max()

In [7]:
# Separando X e y

X = transacoes.drop('Class',axis=1)
y = transacoes.Class

**Separando em treino e teste**

In [9]:
# Separando em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### Realizando o random undersampling

In [11]:
# Definindo o RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

In [12]:
# Definindo a nova amostra

X_resRUS, y_resRUS = rus.fit_resample(X_train, y_train)

**<font color='blue'>Com Regressão Logística**

In [14]:
# Fazendo o fit

clfLog_RUS = LogisticRegression(random_state=42).fit(X_resRUS, y_resRUS)

In [15]:
# Fazendo a previsão

y_predLog_RUS = clfLog_RUS.predict(X_test)

y_pred_probaLog_RUS = clfLog_RUS.predict_proba(X_test)[:,1]

In [16]:
# Visualizando a matriz confusão

confusion_matrix(y_test,y_predLog_RUS)

array([[91301,  2524],
       [   15,   147]], dtype=int64)

In [17]:
# Calculando a área sob a curva precisão x recall

precisionLog_RUS,recallLog_RUS,thresholdsLog_RUS = metrics.precision_recall_curve(y_test,y_pred_probaLog_RUS)
print(metrics.auc(recallLog_RUS, precisionLog_RUS))

0.695159942917231


In [18]:
# Calculando o recall

recall_score(y_test, y_predLog_RUS)

0.9074074074074074

## Vamos criar um novo classificador mudando parâmetros

In [20]:
# Fazendo o novo fit

clfLog_RUS2 = LogisticRegression(random_state=42, solver='newton-cg', C=100).fit(X_resRUS, y_resRUS)

In [21]:
# Fazendo a nova previsão

y_predLog_RUS2 = clfLog_RUS2.predict(X_test)

y_pred_probaLog_RUS2 = clfLog_RUS2.predict_proba(X_test)[:,1]

In [22]:
# Visualizando a nova matriz confusão

confusion_matrix(y_test,y_predLog_RUS2)

array([[90204,  3621],
       [   14,   148]], dtype=int64)

In [23]:
# Traçando a nova área sobre a curva precisão x recall

precisionLog_RUS2,recallLog_RUS2,thresholdsLog_RUS2 = metrics.precision_recall_curve(y_test,y_pred_probaLog_RUS2)
print(metrics.auc(recallLog_RUS2, precisionLog_RUS2))

0.7480857800849602


In [24]:
# Calculando o novo recall

recall_score(y_test, y_predLog_RUS2)

0.9135802469135802

**Podemos então começar a ajustar alguns parâmetros**

**Vamos utilizar o GridSearchCV para automatizar a busca por melhores parâmetros**

In [27]:
# Definindo os parâmetros que queremos testar

parametros = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

In [28]:
# Selecionando a Regressão Logística

LogReg = LogisticRegression(random_state=42)

In [29]:
# Criando um novo classificador usando os parâmetros que escolhemos anteriormente

clf_GS = GridSearchCV(LogReg, parametros, scoring='recall')

In [30]:
# Fazendo o fit dos nossos dados

clf_GS = clf_GS.fit(X_resRUS, y_resRUS)

In [31]:
# Visualizando os melhores parâmetros definidos pelo GridSearchCV 

clf_GS.best_params_

{'C': 0.001, 'solver': 'liblinear'}

In [32]:
# Usando esse modelo para fazer as previsões

y_pred_GS = clf_GS.predict(X_test)

In [33]:
# Analisando a matriz confusão

confusion_matrix(y_test, y_pred_GS)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [34]:
# O recall

recall_score(y_test, y_pred_GS)

0.9382716049382716

In [35]:
# E a precisão

precision_score(y_test, y_pred_GS)

0.017351598173515982

In [36]:
# Podemos visualizar tudo que foi feito

pd.DataFrame(clf_GS.cv_results_)

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.035407      0.010659         0.011556        0.008056   0.001   
1        0.015400      0.006373         0.012673        0.005758   0.001   
2        0.010588      0.003059         0.004717        0.002810   0.001   
3        0.042427      0.008749         0.007196        0.007200   0.001   
4        0.048459      0.005429         0.011973        0.009424   0.001   
5        0.046684      0.009138         0.012426        0.005890    0.01   
6        0.024010      0.005701         0.007430        0.006978    0.01   
7        0.010807      0.008596         0.009170        0.004483    0.01   
8        0.048257      0.004511         0.001396        0.002792    0.01   
9        0.055446      0.007979         0.004852        0.005213    0.01   
10       0.040199      0.005978         0.009380        0.005985     0.1   
11       0.030414      0.013247         0.001352        0.001865     0.1   
12       0.003544      0.006127         0.011820        0.006189     0.1   
13       0.042659      0.005400         0.012145        0.007272     0.1   
14       0.052242      0.008841         0.003537        0.006117     0.1   
15       0.044608      0.009782         0.016269        0.004376       1   
16       0.027231      0.006994         0.010703        0.003932       1   
17       0.010615      0.002457         0.006509        0.001434       1   
18       0.045378      0.002581         0.006670        0.000607       1   
19       0.052382      0.016169         0.014121        0.004050       1   
20       0.115245      0.027165         0.006336        0.005879      10   
21       0.033441      0.002523         0.007186        0.006476      10   
22       0.014901      0.002443         0.001924        0.002366      10   
23       0.045385      0.005155         0.007748        0.001943      10   
24       0.050017      0.008679         0.008984        0.008336      10   
25       0.176595      0.039818         0.009051        0.008390     100   
26       0.051991      0.016895         0.008406        0.007727     100   
27       0.014416      0.004068         0.005855        0.003650     100   
28       0.047885      0.008540         0.004852        0.003966     100   
29       0.050396      0.004781         0.009634        0.005577     100   
30       0.160018      0.046066         0.011931        0.003291    1000   
31       0.055209      0.009413         0.011396        0.005123    1000   
32       0.023663      0.010197         0.003855        0.002678    1000   
33       0.042194      0.011343         0.004423        0.006262    1000   
34       0.049428      0.010622         0.004928        0.006481    1000   

   param_solver                               params  split0_test_score  \
0     newton-cg  {'C': 0.001, 'solver': 'newton-cg'}           0.757576   
1         lbfgs      {'C': 0.001, 'solver': 'lbfgs'}           0.757576   
2     liblinear  {'C': 0.001, 'solver': 'liblinear'}           0.909091   
3           sag        {'C': 0.001, 'solver': 'sag'}           0.757576   
4          saga       {'C': 0.001, 'solver': 'saga'}           0.772727   
5     newton-cg   {'C': 0.01, 'solver': 'newton-cg'}           0.833333   
6         lbfgs       {'C': 0.01, 'solver': 'lbfgs'}           0.833333   
7     liblinear   {'C': 0.01, 'solver': 'liblinear'}           0.863636   
8           sag         {'C': 0.01, 'solver': 'sag'}           0.833333   
9          saga        {'C': 0.01, 'solver': 'saga'}           0.833333   
10    newton-cg    {'C': 0.1, 'solver': 'newton-cg'}           0.863636   
11        lbfgs        {'C': 0.1, 'solver': 'lbfgs'}           0.863636   
12    liblinear    {'C': 0.1, 'solver': 'liblinear'}           0.863636   
13          sag          {'C': 0.1, 'solver': 'sag'}           0.863636   
14         saga         {'C': 0.1, 'solver': 'saga'}           0.863636   
15    newton-cg      {'C': 1, 'solver': 'newton-cg'}           0.86